In [38]:
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px
# renderer for jupyterlab
pio.renderers.default='jupyterlab'
# renderer for vs code
# pio.renderers.default='vscode'
# renderer for colab
# pio.renderers.default='colab'
pio.templates.default = "plotly_white"
import numpy as np
import statsmodels.api as sm
import datetime
from plotly.subplots import make_subplots
from ipywidgets import widgets,Layout
import warnings
warnings.simplefilter(action='ignore')

### Importing the datasets

In [39]:
district=pd.read_csv(r'../data/rainfall-districts.csv')
koeppen=pd.read_csv(r'../data/rainfall-koe-regions.csv')
thornthwaite=pd.read_csv(r'../data/rainfall-tho-regions.csv')
climatic=pd.read_csv(r'../data/rainfall-climatic-regions.csv')

### Rainfall by region type

In [40]:
# dropdown widget
region_type = widgets.Dropdown(
    options=['District','Koeppen Climatic Regions',
             'Thornthwaite Climatic Regions', 'Climatic Region'],
    value='District', description='Region Type',layout=Layout(margin='20px 20px 20px 20px'))
container=widgets.HBox([region_type])

In [41]:
traces=[go.Scatter(x=district['date'],y=district[i],name=i) for i in district.columns[3:]]
layout = go.Layout(title=dict(text='Rainfall by region type'),xaxis=dict(title='Years',rangeslider=dict(visible=True)),yaxis=dict(title='Rainfall'))
go.Layout()
g = go.FigureWidget(data=traces, layout=layout)

In [42]:
def plot_fig(df, title):
    var = {'district': district,
           'koeppen': koeppen,
           'thornthwaite': thornthwaite,
           'climatic': climatic}
    df=var[df]
    with g.batch_update():
        col=list(df.columns.values)
        for i in range(len(g.data)):
            if i+3<len(col):
                g.data[i].y=df[col[i+3]]
                g.data[i].name=col[i+3]
                g.data[i].x=df['date']
            else:
                g.data[i].x=None
                g.data[i].y=None
                g.data[i].name=None

In [43]:
total_container=widgets.VBox([container,g])
display(total_container)

In [44]:
def response(change):
    val=region_type.value.split()[0].lower()
    plot_fig(val,region_type.value)

In [45]:
region_type.observe(response,names='value')

### Rainfall Complete Data

In [46]:
rain=pd.concat([district,koeppen.iloc[:,3:],climatic.iloc[:,3:],thornthwaite.iloc[:,3:]],axis=1)

In [47]:
rain=rain.set_index('date',drop=False)

In [48]:
rain['month']=rain.apply(lambda x: datetime.datetime.strptime(x['date'], "%Y-%m-%d").month,axis=1)
rain['day-of-month']=rain.apply(lambda x: datetime.datetime.strptime(x['date'], "%Y-%m-%d").day,axis=1)
rain=rain[list(rain.columns[:3])+list(rain.columns[48:50])+list(rain.columns[3:48])]

In [49]:
regions=list(rain.columns.values)[5:]

In [53]:
rain.to_csv(r'../data/rainfall-by-region.csv')